# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import cv2
import datetime
import os
from PIL import Image
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers import Dropout, LSTM
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras import regularizers

Using TensorFlow backend.


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./train.csv').readlines())
val_doc = np.random.permutation(open('./val.csv').readlines())

batch_size = 16   #experiment with the batch size

x = 15 #no of images
 
y = 120  #Fixing the image size
z = 120

In [4]:
train_doc = np.random.permutation(open('./train.csv').readlines())
val_doc = np.random.permutation(open('./val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def crop_img(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    return img

#normalize image
def normalize_image(img):
    return (img/255)


In [6]:
def generator_3D(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [frame for frame in range(x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(folder_list)/batch_size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    
                    cropped_image = crop_img(image)
                    resized_image = cv2.resize(cropped_image, (120,120), interpolation = cv2.INTER_CUBIC)
                    normalized_image = normalize_image(resized_image)
                    
                    batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        
        remaining_list = len(folder_list); #56
        if(remaining_list != num_batches*batch_size ):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            num_batches = remaining_list - (batch_size*num_batches)           
            for batch in range(num_batches):
                # we iterate over the number of batches
                batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in

                        image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes

                        cropped_image = crop_img(image)
                        resized_image = cv2.resize(cropped_image, (120,120), interpolation = cv2.INTER_CUBIC)
                        normalized_image = normalize_image(resized_image)

                        batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image

                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [7]:
curr_dt_time = datetime.datetime.now()
train_path = './train'
val_path = './val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 25
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 25


In [8]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

### Model: Conv3D-CNN


In [9]:
num_classes = 5
no_of_Frames= 3
model2 = Sequential()
model2.add(Conv3D(8, (3,3,3), activation='relu', input_shape=(15,120,120,3)))

model2.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model2.add(Conv3D(16, (3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model2.add(Conv3D(32, (3,3,3), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model2.add(Conv3D(64, (2,2,2), activation='relu'))
model2.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model2.add(Flatten())
model2.add(Dense(256))
model2.add(Dropout(0.5))
model2.add(Dense(512))
model2.add(Dropout(0.5))

model2.add(Dense(num_classes, activation='softmax'))


### Model 2 (Conv3D-CNN) Summary

In [10]:

optimiser = optimizers.SGD(lr=0.01,nesterov=True)
model2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 13, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 13, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 11, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 11, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 9, 26, 26, 32)     13856     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 9, 13, 13, 32)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 8, 12, 12, 64)     16448     
__________

In [11]:
train_generator = generator_3D(train_path, train_doc, batch_size)
val_generator = generator_3D(val_path, val_doc, batch_size)

In [12]:
model_name = 'Conv3D_CNN' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.02, verbose=1, patience=3)  # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [13]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./val ; batch size = 16
Source path =  ./train ; batch size = 16
Epoch 1/25
Batch:  7 Index: 16
42/42 [==============================] - 25s 589ms/step - loss: 1.6016 - categorical_accuracy: 0.2411 - val_loss: 1.5707 - val_categorical_accuracy: 0.2679

Epoch 00001: val_loss improved from inf to 1.57072, saving model to Conv3D_CNN_2020-05-0407_40_21.657448/model-00001-1.60158-0.24107-1.57072-0.26786.h5
Epoch 2/25
42/42 [==============================] - 22s 530ms/step - loss: 1.5960 - categorical_accuracy: 0.2143 - val_loss: 1.5831 - val_categorical_accuracy: 0.4196

Epoch 00002: val_loss did not improve from 1.57072
Epoch 3/25
42/42 [==============================] - 24s 563ms/step - loss: 1.5709 - categorical_accuracy: 0.2634 - val_loss: 1.5287 - val_categorical_accuracy: 0.3036

Epoch 00003: val_loss improved from 1.57072 to 1.52872, saving model to Conv3D_CNN_2020-05-0407_40_21.657448/model-00003-1.57089-0.26339-1.52872-0.30357.h5
Epoch 4/25
42/42 [===================

42/42 [==============================] - 22s 526ms/step - loss: 0.7207 - categorical_accuracy: 0.7470 - val_loss: 0.7990 - val_categorical_accuracy: 0.7143

Epoch 00019: val_loss improved from 0.84642 to 0.79898, saving model to Conv3D_CNN_2020-05-0407_40_21.657448/model-00019-0.72072-0.74702-0.79898-0.71429.h5
Epoch 20/25
42/42 [==============================] - 23s 550ms/step - loss: 0.7278 - categorical_accuracy: 0.7426 - val_loss: 1.7037 - val_categorical_accuracy: 0.4196

Epoch 00020: val_loss did not improve from 0.79898
Epoch 21/25
42/42 [==============================] - 23s 542ms/step - loss: 0.6726 - categorical_accuracy: 0.7560 - val_loss: 0.7540 - val_categorical_accuracy: 0.6964

Epoch 00021: val_loss improved from 0.79898 to 0.75399, saving model to Conv3D_CNN_2020-05-0407_40_21.657448/model-00021-0.67256-0.75595-0.75399-0.69643.h5
Epoch 22/25
42/42 [==============================] - 22s 534ms/step - loss: 0.5891 - categorical_accuracy: 0.7768 - val_loss: 0.9748 - val_cat

##### categorical_accuracy: 0.8631
##### val_categorical_accuracy: 0.7411
